In [15]:
import pandas as pd

## Exercícios

1. Considere a existência de três tabelas distintas:
* customer.csv : Possui a informação dos clientes em duas colunas: customer id  customer name
* products.csv : Conté informação dos produtos vendidos pela empresa em três colunas - p_id (product id), product (name) e price
* sales.csv : Contém informações das vendas realizadas em seis colunas - sale_id, c_id (customer id), p_id (product_id), qty (quantity sold), store (name)

Conhecendo as bases e utilizando os métodos de concatenação de bases responda:


a) Quais produtos não foram vendidos?

In [ ]:
import pandas as pd 
sales = pd.read_csv("data/sales.csv") 
products = pd.read_csv("data/products.csv") 

# Faz um merge priorizando as informações da tabela de produtos
my_data = pd.merge(sales, products, on=['p_id','product'],how='right')
 
# Seleciona todos as linhas em que o sale_id é nan
my_data = my_data[my_data['sale_id'].isna()]

# Seleciona os produtos que não foram vendidos
print(my_data.loc[:,'product'])

b) Quantos clientes não realizaram uma compra? 

In [ ]:
customers = pd.read_csv("data/customer.csv") 

# Faz um merge priorizando as informações da tabela de customers
my_data = pd.merge(sales, customers, on='c_id', how='right')

# Seleciona todos as linhas em que o sale_id é nan
my_data = my_data[my_data['sale_id'].isna()] 

# Seleciona os clientes que não realizaram compras
print(my_data.loc[:,'Customer']) 

c) Liste a quantidade vendida e o faturamento de cada produto 

In [ ]:
my_sum=pd.merge(sales, products, how='left', on=['p_id','product'])

# Calcula valor total de cada sale_id
my_sum['total_sale'] = my_sum['qty']*my_sum['price']

# Agrupa por produto e soma a quantidade vendida e o valor total da compra
my_sum.groupby(['product'])[['qty', 'total_sale']].sum()

d) Liste a quantidade vendida de cada produto por loja

In [ ]:
my_sale = sales.groupby(['product','p_id', 'store'])[['qty']].sum().reset_index()
my_sale

e) Qual loja teve maior faturamento?

In [ ]:
my_sum.groupby(['store'])[['total_sale']].sum().sort_values('total_sale').tail(1)

f) Qual produto foi o mais vendido?

In [ ]:
my_sum.groupby(['product'])[['qty']].sum().sort_values('qty').tail(1)

## Referências
https://pandas.pydata.org/docs/user_guide/groupby.html <br>
https://pandas.pydata.org/docs/user_guide/merging.html <br> 
https://towardsdatascience.com/python-pandas-dataframe-join-merge-and-concatenate-84985c29ef78 <br>
[When to use pandas transform function](https://towardsdatascience.com/when-to-use-pandas-transform-function-df8861aa0dcf) <br>
[Compara a performance entre várias formas de iterar em um df. Vai desde o for até apply e transform](https://youtu.be/rsyvErL0Bo8) <br>

## Material extra

### Outros parâmetros do groupby por default
* as_index
* sort
* dropna # exclui nans nas keys

<br> Em todas o default do python é True <br>
df.groupby('Pclass', sort=False)["Fare"].mean()

In [ ]:
# dropna
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

In [ ]:
# Default ``dropna`` is set to True, which will exclude NaNs in keys
df_dropna.groupby(by=["b"], dropna=True).sum()

In [ ]:
df_dropna.groupby(by=["b"], dropna=False).sum()

Repare que podemos chamar qualquer função do `pd.Series` ou  do `numpy`

In [ ]:
df.groupby(["Survived"]).mean()

### Função Lambda
Uma função lambda nada mais é que uma **forma alternativa de declarar uma função**, de um jeito mais direto

In [ ]:
# função que retorna o dobro de um número usando def
def dobro(x):
    
    return 2*x

dobro(2)

In [ ]:
# função que retorna o dobro de um número usando lambda x
faz_dobro = lambda x: 2*x

In [ ]:
faz_dobro(6)

### Apply
O método `.apply()` recebe uma função como input e aplica ela para todo o df como se fosse um loop. Se você quiser que essa função seja aplicada ao longo das colunas deve considerar axis=0 e ao longo das linhas axis=1)

In [ ]:
df.groupby(['Pclass']).apply(lambda x: x.describe())

Uma grande funcionalidade do pandas é que com o método `apply()` podemos aplicar uma **função** (muitas vezes, uma **função lambda**) a uma coluna ou linha de um DataFrame



Vamos selecionar a coluna de idades...

In [ ]:
df["Age"]

Aplicando uma função lambda **a todos os elementos da coluna**, ou seja, **à todas as linhas da tabela, daquela coluna específica**:

Tomando cada idade + 2, usando a função lambda definida.

Essa função lambda é equivalente a:

```python

def funcao(x):

    return x + 2
```

In [ ]:
df["Age"].apply(lambda x: x + 2)

In [ ]:
def funcao(x):
    return x + 2

df.Age.apply(funcao)

In [ ]:
df.Age.transform(funcao)

Um outro exemplo:

In [ ]:
# função: transforma todos os números em string, e concatena "!!!!!!!!!" à string
df["Age"].apply(lambda x: str(x) + "!!!!!!!!!")

Vamos usar uma função lambda para **extrair o sobrenome** dos nomes dos passageiros

Pra extrarir o sobrenome, note que este está separada do resto do nome por vírgula.

Para perceber isso, dê uma olhada na coluna de nomes:

In [ ]:
df["Name"]

Portanto, podemos usar a função para strings `split(",")`, com quebra na vírgula, e depois selecionar o primeiro elemento da lista gerada!

Vamos aproveitar e **criar uma nova coluna da base**, com os sobrenomes!

In [ ]:
df["Surname"] = df["Name"].apply(lambda x: x.split(",")[0])

In [ ]:
df["Surname"]

### Apply com funções

E se quisessemos comparar o quanto cada passageiro pagou a mais ou a menos da média do Fare?

In [ ]:
def f(group):
    return pd.DataFrame({'Fare_original': group,
                         'Fare_variacao': group - group.mean()})

df[['Fare_original','Fare_variacao']] = df.groupby('Pclass')['Fare'].apply(f)

In [ ]:
df.head()

#### Transform X Apply
Com uma função de agregação o `.transform()` retorna um df que tem a mesma quantidade de linhas que o df original enquanto o `.apply` retorna o agregado por grupos.

### Filtros
O filtro retorna apenas um subset do nosso df. Aqui podemos aplicar filtros mais elaborados do que os vistos na última aula. <br>
Podemos, por exemplo, eliminar categorias do df que possuem apenas alguns elementos:

In [ ]:
df.SibSp.value_counts()

In [ ]:
df.shape

In [ ]:
def filter_func(x):
    return x['Fare'] - x.Fare_Mean < 100

# df_filter = df.groupby(['SibSp']).filter(lambda x: filter_func(x))

df_filter = df.groupby(['SibSp']).filter(lambda x: len(x) >20)
df_filter.shape

In [ ]:
df_filter.SibSp.value_counts()

Vamos supor que antes de afundar o titanic, o time de hapiness quisesse promover uma jogatina para os grupos (segmentado por classe e sexo) que possuem idade média acima de 30 anos.

In [ ]:
df.groupby(['Pclass','Sex'])[['Age']].mean()

como podemos filtrar nosso df para termos apenas os passageiros que pertecem a essas segmentações escolhidas?

In [ ]:
df.groupby(['Pclass','Sex']).filter(lambda x: x['Age'].mean()>30)